## Google Dance

In [1]:
#Importing all the libraries required for scraping

from splinter import Browser
from bs4 import BeautifulSoup 
import requests
import time
import pandas as pd
import pprint
import webbrowser
import urllib
import re
import pymongo
from geopy.geocoders import Nominatim  

In [2]:
# using the crome driver for scraping

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
#browser = Browser()

In [3]:
# Url for main page
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=Y5HcXZW5CdG05gKT7KzIAw&q=kids+dance+classes+new+jersey&oq=kids+dance+classes+new+jersey&gs_l=psy-ab.3...140911.143275.0.143677.6.6.0.0.0.0.271.904.0j4j1.5.0....0...1c.1.64.psy-ab..1.0.0....0.2IUp-hMGy8U#rlfi=hd:;si:;mv:[[41.0322952,-73.93508229999999],[40.534548400000006,-74.637359]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!1m4!1u2!2m2!2m1!1e1!2m1!1e2!2m1!1e3!3sIAE,lf:1,lf_ui:2'
browser.visit(url)

# updating the page by clicking on search button
browser.find_by_name('btnG').first.click()

# parser
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

# storing all the data from the main class
Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
# search variables Name and reviews within main class using for loop
for tr in Data:
    dataframe = {}
    dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

22
[{'name': 'Hudson Dance Studio Edgewater'}, {'name': 'NJ School of Ballet'}, {'name': 'Wild Child Dance Studio'}, {'name': 'The Kennedy Dancers'}, {'name': 'Hudson Dance Studio Edgewater'}, {'name': 'bboy bgirl lifestyle'}, {'name': 'Next Step Broadway'}, {'name': 'Performing Arts Workshop'}, {'name': 'Nimbus Dance'}, {'name': 'Salsa Fever On 2 Dance Academy'}, {'name': 'Class Act Performing Arts Studio'}, {'name': 'Dance Palooza Studios'}, {'name': 'My Passion Dance Studio & Entertainment'}, {'name': 'Monroe Street Movement Space'}, {'name': 'Dance Power'}, {'name': 'Harmony Dance Center'}, {'name': 'Jersey City Ballet'}, {'name': 'Union City Music & Dance Academy'}, {'name': 'Garden Street School of the Performing Arts'}, {'name': 'Starlight Dance Center'}, {'name': 'Uptown Dance Studio'}, {'name': 'NJ Salsa Dancing Lessons'}]


In [4]:

Data = []
# Using the geo locator for getting Lat and Long
geolocator = Nominatim(user_agent="my-application")

# getting the link for next pages 

pageData = soup.find_all(class_='std')

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    print(pLink)

#updating each page link with specific nunmbers to go to next page

top_result = []
list1 = [2, 4, 6,8,10,12,14,16,18] 
for x in (list1):
    prePage = x
    #print(prePage)
    startPage = "start="+ str(prePage) +"0"
    #print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    #print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    #print(pageLink)
    browser.visit(pageLink)

# parsing pages as the pages updated with above url

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    #print(len(Data))

# search variables Name, url, adress, phone no, zipcode within main class using for loop

    for tr in Data:
        
        dataframe = {}
        dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["type"] = "Dance Classes"
        
#if the url is empty handling the exception

        try:
            dataframe["url"] = (tr.find("a",attrs = {"class": "yYlJEf L48Cpd"})['href'])
            
            #print(dataframe["url"])
        except:
            print('Cant Find the url')

# scraping for phone number

        PhoneData = tr.find('span').text.split(' ')
        #print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        
# if the main directory is empty, handling the exception
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
        
# getting the address
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])

# cleaning the address based on number of delimters

            orgAddress = rawAddress.split('/')[4].strip()
            #print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            #print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[1]+', ' + orgAddress[2]+', ' + ZipCode[1]+', ' + ZipCode[2]
                #dataframe["city"] = orgAddress[2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-3]+', '+orgAddress[lenAddress-2]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-2]+', '+orgAddress[lenAddress-1]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-1]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                #    print(orgAddress)
                
# Using geolocator to find the lat and long for each address

            try:
                locForLatLong = dataframe["address"] #+ dataframe["city"]
                #print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                #print(location.address)
                #print((location.latitude,location.longitude))
                dataframe["location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
                
        top_result.append(dataframe)

#print(top_result)

# creating a dataframe for Dance
df1=pd.DataFrame(top_result)



1
/search?q=kids+dance+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=GPzrXYS7K6Gp_QbGz7OwCA&start=20&sa=N&ved=0ahUKEwiEp8C7oaTmAhWhVN8KHcbnDIYQ8tMDCOgC
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the url
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the url
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the url
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the url
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the url
Cant Find the address
Cant Find the address
Cant Find the addr

In [5]:
#df1.to_excel("outputDance.xlsx")

# creating a dataframe for Dance
df1


,address,location,name,phoneNo,type,url
0,"372 9th St Unit B, Jersey City, NJ, 07302",NaN,Jersey City Muay Thai,,Dance Classes,http://www.jerseycitymuaythai.com/
1,"780 Salem Ave, Elizabeth, NJ, 07208","[40.6813924411765, -74.2172814705882]",The Institute of Music for Children,,Dance Classes,http://www.instituteofmusic.org/
2,"99 W Madison Ave, Dumont, NJ, 07628","[40.94016, -73.9985329090909]",Broadway Bound Dance Center Inc.,,Dance Classes,http://www.bergencountydanceclasses.com/
3,"357 Broad St %231, Bloomfield, NJ, 07003",NaN,Broadway Performing Arts,9737482787,Dance Classes,http://broadwayperformingarts.net/
4,"234 Franklin Ave, Nutley, NJ, 07110","[40.815737, -74.161944]",Ellis Dance Studio,,Dance Classes,http://ellisdancestudios.com/
5,"355 Eisenhower Pkwy %23100, Livingston, NJ, ...",NaN,Grooves Unlimited Dance Studio,,Dance Classes,http://www.groovesunlimiteddancestudio.com/
6,"15-01 Pollitt Dr, Fair Lawn, NJ, 07410","[40.9447716, -74.1260103]",Dance Designs Studio,,Dance Classes,http://dancedesignsstudio.com/
7,"300 Observer Hwy, Hoboken, NJ, 07030","[40.736303, -74.035171]",Believe Ballroom Dance Hoboken,,Dance Classes,http://www.bienvenueballroom.com/
8,"2 Villa Dr, Edison, NJ, 08820","[40.599712, -74.354303]",Edison School of Music and Dance,,Dance Classes,http://www.edisonmusicanddance.com/
9,"109 Monroe St Ste 201, Newark, NJ, 07105",NaN,Luna Mia Dance Studio,,Dance Classes,http://www.lunamiadance.com/


## Google Music

In [6]:
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&tbm=lcl&sxsrf=ACYBGNTdFaDd4jAeC6lZKeILHtxz3pMx4A%3A1574389394556&ei=kkbXXanHIeSV_Qbz-a_wAg&q=kids+music+classes+new+jersey&oq=kids+music+classes+new+jersey&gs_l=psy-ab.12...0.0.0.163915.0.0.0.0.0.0.0.0..0.0....0...1c..64.psy-ab..0.0.0....0.51R-_vCUfl4#rlfi=hd:;si:;mv:[[41.0328425,-73.9399237],[40.1005597,-74.3127616]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!1m4!1u2!2m2!2m1!1e1!1m4!1u16!2m2!16m1!1e1!1m4!1u16!2m2!16m1!1e2!2m1!1e2!2m1!1e3!2m1!1e16!3sIAE,lf:1,lf_ui:2'
browser.visit(url)

browser.find_by_name('btnG').first.click()

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
for tr in Data:
    dataframe = {}
    dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

22
[{'name': 'World of ABC Preschool'}, {'name': 'Rockness Music of Montclair'}, {'name': 'Rockness Music of Jersey City'}, {'name': 'The Institute of Music for Children'}, {'name': 'Little Rockers - Cool Kids Music Classes'}, {'name': 'Little Rockers - cool kids music classes & parties'}, {'name': 'A Sound Start'}, {'name': 'Little Rockers - cool kids music classes & parties'}, {'name': 'Music Together of Bergen County'}, {'name': 'Musical Munchkins of New Jersey'}, {'name': 'Jersey City Music Lessons'}, {'name': 'New City Kids'}, {'name': 'Music Play Studios'}, {'name': 'Joy Kids Music & Art (Joy Performing Art Center)'}, {'name': 'School of Rock Hoboken'}, {'name': 'Union City Music & Dance Academy'}, {'name': 'Rockness Music of Hoboken (Midtown)'}, {'name': 'Jersey City Art School'}, {'name': "Kids' MusicRound"}, {'name': 'Rockness Music of West New York'}, {'name': 'Arete Music Academy'}, {'name': 'Music Together of Montclair'}]


In [7]:
Data = []
geolocator = Nominatim(user_agent="my-application")
pageData = soup.find_all(class_='std')

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    print(pLink)

top_result = []
list1 = [2, 4, 6,8,10,12,14,16,18] 
for x in (list1):
    prePage = x
    #print(prePage)
    startPage = "start="+ str(prePage) +"0"
    #print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    #print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    #print(pageLink)
    browser.visit(pageLink)

  
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    #print(len(Data))
    
    for tr in Data:
        
        dataframe = {}
        dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["type"] = "Music Classes"
        try:
            dataframe["url"] = (tr.find("a",attrs = {"class": "yYlJEf L48Cpd"})['href'])
            
            #print(dataframe["url"])
        except:
            print('Cant Find the url')
            
        
        PhoneData = tr.find('span').text.split(' ')
        #print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])
            
            orgAddress = rawAddress.split('/')[4].strip()
            #print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            #print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[1]+', ' + orgAddress[2]+', ' + ZipCode[1]+', ' + ZipCode[2]
                #dataframe["city"] = orgAddress[2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-3]+', '+orgAddress[lenAddress-2]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-2]+', '+orgAddress[lenAddress-1]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-1]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
                    #print(orgAddress)
            try:
                locForLatLong = dataframe["address"] #+ dataframe["city"]
                #print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                #print(location.address)
                #print((location.latitude,location.longitude))
                dataframe["location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
                
        top_result.append(dataframe)

#print(top_result)
df2=pd.DataFrame(top_result)



1
/search?q=kids+music+classes+new+jersey&rlz=1C1CHBF_enUS853US853&tbm=lcl&ei=g_zrXZzYMcO8ggekyL7gDQ&start=20&sa=N&ved=0ahUKEwjcpcnuoaTmAhVDnuAKHSSkD9wQ8tMDCIcC
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find the url
Cant Find t

In [8]:
#df.to_excel("outputMusic.xlsx")

# Creating a dataframe for Music
df2

,name,phoneNo,type
0,Guitar Bar Jr.,,Music Classes
1,Music for Aardvarks,,Music Classes
2,Kidville,930,Music Classes
3,Kids Party Art,,Music Classes
4,School of Rock Saddle Brook,,Music Classes
5,Art Kids Academy,,Music Classes
6,Kindermusik with Susan Leone,,Music Classes
7,Sharon's Studio of Dance & Music,1030,Music Classes
8,Kids' Music Round,,Music Classes
9,School of Rock Montclair,,Music Classes


## Google Sports

In [9]:
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=05bdXaEyoqX9BoGbkuAK&q=kids+sports+classes+new+jersey&oq=kids+sports+classes+new+jersey&gs_l=psy-ab.12...13265.16654.0.22362.7.7.0.0.0.0.83.504.7.7.0....0...1c.1.64.psy-ab..0.0.0....0.EEUl6U13Yy0#rlfi=hd:;si:;mv:[[41.1122716,-73.93472659999999],[40.3136717,-74.6914486]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!2m1!1e3!3sIAE,lf:1,lf_ui:3'
browser.visit(url)

browser.find_by_name('btnG').first.click()

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
for tr in Data:
    dataframe = {}
    dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

22
[{'name': 'Camp Pinebrook'}, {'name': 'Learn On The Ark'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'New Jersey Kick Boxing'}, {'name': 'New York Sports Clubs'}, {'name': 'New York Sports Clubs'}, {'name': 'My Gym Jersey City'}, {'name': 'New York Sports Clubs'}, {'name': 'I9 Sports'}, {'name': 'Jersey City Muay Thai'}]


In [10]:
Data = []
geolocator = Nominatim(user_agent="my-application")
pageData = soup.find_all(class_='std')
#print( soup.find("div", class_="std")["aria-label"] )

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    #print(pLink)

top_result = []
list1 = [2, 4, 6,8,10,12] 
for x in (list1):
    prePage = x
    #print(prePage)
    startPage = "start="+ str(prePage) +"0"
    #print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    #print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    #print(pageLink)
    browser.visit(pageLink)

  
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    #print(len(Data))
    
    for tr in Data:
        
        dataframe = {}
        dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["type"] = "Sports Classes"
        try:
            dataframe["url"] = (tr.find("a",attrs = {"class": "yYlJEf L48Cpd"})['href'])
            
            #print(dataframe["url"])
        except:
            print('Cant Find the url')
            
        
        PhoneData = tr.find('span').text.split(' ')
        #print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])
            
            orgAddress = rawAddress.split('/')[4].strip()
            #print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            #print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[1]+', ' + orgAddress[2]+', ' + ZipCode[1]+', ' + ZipCode[2]
                #dataframe["city"] = orgAddress[2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-3]+', '+orgAddress[lenAddress-2]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                #print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-2]+', '+orgAddress[lenAddress-1]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-1]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                    #print(orgAddress)
                    
            try:
                locForLatLong = dataframe["address"]# + dataframe["city"]
                #print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                #print(location.address)
                #print((location.latitude,location.longitude))
                dataframe["location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
        top_result.append(dataframe)

#print(top_result)
df3=pd.DataFrame(top_result)



1
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the url
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the address
Cant Find the url
Cant Find the address
Cant Find the address
Cant Find the address


In [11]:
#df.to_excel("outputSports.xlsx")

# creating a dataframe for sports
df3


,address,location,name,phoneNo,type,url
0,"459 NJ-17, Hasbrouck Heights, NJ, 07604","[40.8643850620946, -74.0646431213973]",SafeSplash Swim School - Hasbrouck Heights,3,Sports Classes,https://www.safesplash.com/locations/hasbrouck...
1,"69 NJ-17, Hasbrouck Heights, NJ, 07604","[40.852465341022, -74.0734070002573]",Meadowlands Gymnastics Academy,230,Sports Classes,http://meadowlandsgym.com/
2,"1121 NJ-34 Suite O, Matawan, NJ, 07747",NaN,The Little Gym of Aberdeen,7327720060,Sports Classes,http://www.thelittlegym.com/aberdeennj
3,"434 Broadway, Bayonne, NJ, 07002","[40.6617075, -74.1200142]",Bayonne Karate For Kids,3,Sports Classes,http://parkkarate.com/
4,NaN,NaN,TGA Premier Sports of Bergen County,9,Sports Classes,http://www.playtga.com/bergencounty
5,"150 Newark Ave, Jersey City, NJ, 07302","[40.720706, -74.0439375360249]",JC Taekwondo and Kickboxing Academy,3,Sports Classes,http://www.jctkd.com/
6,"720 Monroe St, Hoboken, NJ, 07030","[40.7466675918367, -74.0378767142857]",Monroe Street Movement Space,9,Sports Classes,http://www.movementspace.com/
7,"143 Coe Ave, Hillside, NJ, 07205","[40.6910877663854, -74.2200071682221]",i9 Sports,,Sports Classes,http://www.i9sports.com/
8,"2040 Springdale Rd, Cherry Hill, NJ, 08003","[39.9120628, -74.9656516]",Kid Fit NJ,9,Sports Classes,https://www.nj-kidfit.com/
9,"53 Vervalen St, Closter, NJ, 07624","[40.9704506177326, -73.9582628723433]",Kidville,2019831551,Sports Classes,http://www.kidville.com/closter


## Merged all three dataframe into one DF

In [12]:
# merging all 3 dataframe for music, Dannce and Sports

df = df1.append(df2, ignore_index=True)
df = df.append(df3, ignore_index=True)

# Cleaning the dataframet to drop null values for location

df = df.dropna(axis=0, subset=['location','url'])
df = df[df.location != 0]
df

C:\Users\mdegu\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,address,location,name,phoneNo,type,url
1,"780 Salem Ave, Elizabeth, NJ, 07208","[40.6813924411765, -74.2172814705882]",The Institute of Music for Children,,Dance Classes,http://www.instituteofmusic.org/
2,"99 W Madison Ave, Dumont, NJ, 07628","[40.94016, -73.9985329090909]",Broadway Bound Dance Center Inc.,,Dance Classes,http://www.bergencountydanceclasses.com/
4,"234 Franklin Ave, Nutley, NJ, 07110","[40.815737, -74.161944]",Ellis Dance Studio,,Dance Classes,http://ellisdancestudios.com/
6,"15-01 Pollitt Dr, Fair Lawn, NJ, 07410","[40.9447716, -74.1260103]",Dance Designs Studio,,Dance Classes,http://dancedesignsstudio.com/
7,"300 Observer Hwy, Hoboken, NJ, 07030","[40.736303, -74.035171]",Believe Ballroom Dance Hoboken,,Dance Classes,http://www.bienvenueballroom.com/
8,"2 Villa Dr, Edison, NJ, 08820","[40.599712, -74.354303]",Edison School of Music and Dance,,Dance Classes,http://www.edisonmusicanddance.com/
10,"1 Pierson Rd, Maplewood, NJ, 07040","[40.724692, -74.279572]",Performers Theatre Workshop,,Dance Classes,http://ptwonline.com/
11,"37 Gill Ln, Iselin, NJ, 08830","[40.5689118285714, -74.3237624857143]",Nritya Creations Academy of Dance,,Dance Classes,http://www.nrityacreations.com/
12,"496 Inman Ave, Rahway, NJ, 07065","[40.6001789, -74.3113377]",Dance Starz Academy of NJ,,Dance Classes,http://dancestarznj.net/
14,"4500 Park Ave, Weehawken, NJ, 07086","[40.7758993, -74.0195757]",JZT Dance & Yoga,,Dance Classes,http://www.jztdanceandyoga.com/


In [13]:
# exporting to excel

df.to_excel("outputKidsActivities.xlsx") 

In [14]:
# reseting the index

df=df.reset_index(drop=True)
df.head()

,address,location,name,phoneNo,type,url
0,"780 Salem Ave, Elizabeth, NJ, 07208","[40.6813924411765, -74.2172814705882]",The Institute of Music for Children,,Dance Classes,http://www.instituteofmusic.org/
1,"99 W Madison Ave, Dumont, NJ, 07628","[40.94016, -73.9985329090909]",Broadway Bound Dance Center Inc.,,Dance Classes,http://www.bergencountydanceclasses.com/
2,"234 Franklin Ave, Nutley, NJ, 07110","[40.815737, -74.161944]",Ellis Dance Studio,,Dance Classes,http://ellisdancestudios.com/
3,"15-01 Pollitt Dr, Fair Lawn, NJ, 07410","[40.9447716, -74.1260103]",Dance Designs Studio,,Dance Classes,http://dancedesignsstudio.com/
4,"300 Observer Hwy, Hoboken, NJ, 07030","[40.736303, -74.035171]",Believe Ballroom Dance Hoboken,,Dance Classes,http://www.bienvenueballroom.com/


In [15]:
df=df.reset_index()
df.head()

,index,address,location,name,phoneNo,type,url
0,0,"780 Salem Ave, Elizabeth, NJ, 07208","[40.6813924411765, -74.2172814705882]",The Institute of Music for Children,,Dance Classes,http://www.instituteofmusic.org/
1,1,"99 W Madison Ave, Dumont, NJ, 07628","[40.94016, -73.9985329090909]",Broadway Bound Dance Center Inc.,,Dance Classes,http://www.bergencountydanceclasses.com/
2,2,"234 Franklin Ave, Nutley, NJ, 07110","[40.815737, -74.161944]",Ellis Dance Studio,,Dance Classes,http://ellisdancestudios.com/
3,3,"15-01 Pollitt Dr, Fair Lawn, NJ, 07410","[40.9447716, -74.1260103]",Dance Designs Studio,,Dance Classes,http://dancedesignsstudio.com/
4,4,"300 Observer Hwy, Hoboken, NJ, 07030","[40.736303, -74.035171]",Believe Ballroom Dance Hoboken,,Dance Classes,http://www.bienvenueballroom.com/


In [16]:
df = df.rename(columns={'index':'_id'})
df.head()

,_id,address,location,name,phoneNo,type,url
0,0,"780 Salem Ave, Elizabeth, NJ, 07208","[40.6813924411765, -74.2172814705882]",The Institute of Music for Children,,Dance Classes,http://www.instituteofmusic.org/
1,1,"99 W Madison Ave, Dumont, NJ, 07628","[40.94016, -73.9985329090909]",Broadway Bound Dance Center Inc.,,Dance Classes,http://www.bergencountydanceclasses.com/
2,2,"234 Franklin Ave, Nutley, NJ, 07110","[40.815737, -74.161944]",Ellis Dance Studio,,Dance Classes,http://ellisdancestudios.com/
3,3,"15-01 Pollitt Dr, Fair Lawn, NJ, 07410","[40.9447716, -74.1260103]",Dance Designs Studio,,Dance Classes,http://dancedesignsstudio.com/
4,4,"300 Observer Hwy, Hoboken, NJ, 07030","[40.736303, -74.035171]",Believe Ballroom Dance Hoboken,,Dance Classes,http://www.bienvenueballroom.com/


In [17]:
# Add 1 to each "_id" because first activity must start with 1 not 0.  Starting with "_id" = 0 will throw an error
df['_id'] = df['_id'] + 1 
df

,_id,address,location,name,phoneNo,type,url
0,1,"780 Salem Ave, Elizabeth, NJ, 07208","[40.6813924411765, -74.2172814705882]",The Institute of Music for Children,,Dance Classes,http://www.instituteofmusic.org/
1,2,"99 W Madison Ave, Dumont, NJ, 07628","[40.94016, -73.9985329090909]",Broadway Bound Dance Center Inc.,,Dance Classes,http://www.bergencountydanceclasses.com/
2,3,"234 Franklin Ave, Nutley, NJ, 07110","[40.815737, -74.161944]",Ellis Dance Studio,,Dance Classes,http://ellisdancestudios.com/
3,4,"15-01 Pollitt Dr, Fair Lawn, NJ, 07410","[40.9447716, -74.1260103]",Dance Designs Studio,,Dance Classes,http://dancedesignsstudio.com/
4,5,"300 Observer Hwy, Hoboken, NJ, 07030","[40.736303, -74.035171]",Believe Ballroom Dance Hoboken,,Dance Classes,http://www.bienvenueballroom.com/
5,6,"2 Villa Dr, Edison, NJ, 08820","[40.599712, -74.354303]",Edison School of Music and Dance,,Dance Classes,http://www.edisonmusicanddance.com/
6,7,"1 Pierson Rd, Maplewood, NJ, 07040","[40.724692, -74.279572]",Performers Theatre Workshop,,Dance Classes,http://ptwonline.com/
7,8,"37 Gill Ln, Iselin, NJ, 08830","[40.5689118285714, -74.3237624857143]",Nritya Creations Academy of Dance,,Dance Classes,http://www.nrityacreations.com/
8,9,"496 Inman Ave, Rahway, NJ, 07065","[40.6001789, -74.3113377]",Dance Starz Academy of NJ,,Dance Classes,http://dancestarznj.net/
9,10,"4500 Park Ave, Weehawken, NJ, 07086","[40.7758993, -74.0195757]",JZT Dance & Yoga,,Dance Classes,http://www.jztdanceandyoga.com/


In [18]:
# adding to directory
split_data = df.to_dict('records')
split_data

[{'_id': 1,
  'address': ' 780 Salem Ave,  Elizabeth, NJ, 07208',
  'location': [40.6813924411765, -74.2172814705882],
  'name': 'The Institute of Music for Children',
  'phoneNo': '',
  'type': 'Dance Classes',
  'url': 'http://www.instituteofmusic.org/'},
 {'_id': 2,
  'address': ' 99 W Madison Ave,  Dumont, NJ, 07628',
  'location': [40.94016, -73.9985329090909],
  'name': 'Broadway Bound Dance Center Inc.',
  'phoneNo': '',
  'type': 'Dance Classes',
  'url': 'http://www.bergencountydanceclasses.com/'},
 {'_id': 3,
  'address': ' 234 Franklin Ave,  Nutley, NJ, 07110',
  'location': [40.815737, -74.161944],
  'name': 'Ellis Dance Studio',
  'phoneNo': '',
  'type': 'Dance Classes',
  'url': 'http://ellisdancestudios.com/'},
 {'_id': 4,
  'address': ' 15-01 Pollitt Dr,  Fair Lawn, NJ, 07410',
  'location': [40.9447716, -74.1260103],
  'name': 'Dance Designs Studio',
  'phoneNo': '',
  'type': 'Dance Classes',
  'url': 'http://dancedesignsstudio.com/'},
 {'_id': 5,
  'address': ' 300 

In [19]:
# Ensure that all types are objects
df.dtypes

_id          int64
address     object
location    object
name        object
phoneNo     object
type        object
url         object
dtype: object

In [20]:
# Set up the MongoDB connection through pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

# Create DB

db = myclient["KidsRidedb3"]

# Create collection and insert all the data into the MongoDB

mycol = db["activityData21"]
mycol.drop()
mycol.insert_many(split_data)